In [2]:
# import package

# model
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

# dataset and transformation
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision import models

# display images
from torchvision import utils
import matplotlib.pyplot as plt
%matplotlib inline

# utils
import numpy as np
from glob import glob
import os
from tqdm import tqdm

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f'Using {device} for inference')

Using cpu for inference


In [3]:
# train test split

import csv
import shutil
import pandas as pd

train_list = []
test_list = []
test_views=[]
train_views=[]

train_csv=pd.read_csv('train.csv',index_col=0)
train_list=train_csv['video_id'].values.tolist()
train_views=train_csv['views'].values.tolist()

test_csv=pd.read_csv('test.csv',index_col=0)
test_list=test_csv['video_id'].values.tolist()
test_views=test_csv['views'].values.tolist()
#full_data_path = './data/'



print(f'csv len: {len(test_list) + len(train_list)}')
print("train len: ", len(train_list), "\ntest len: ", len(test_list))
# print(f'full data len: {len(os.listdir("./data"))}')
'''
for test in tqdm(test_list):
  if os.path.exists('./medium_15287/' + test + '.jpg'):  
    img_path = glob('./medium_15287/' + test + '.jpg')[0]
    shutil.copyfile(img_path, './test/' + test + '.jpg')
  
  else:
    print(test)

for train in tqdm(train_list):
  if os.path.exists('./medium_15287/' + train + '.jpg'):  
    img_path = glob('./medium_15287/' + train + '.jpg')[0]
    shutil.copyfile(img_path, './train/' + train + '.jpg')
  else:
    print(train)
'''



csv len: 15287
train len:  13758 
test len:  1529


"\nfor test in tqdm(test_list):\n  if os.path.exists('./medium_15287/' + test + '.jpg'):  \n    img_path = glob('./medium_15287/' + test + '.jpg')[0]\n    shutil.copyfile(img_path, './test/' + test + '.jpg')\n  \n  else:\n    print(test)\n\nfor train in tqdm(train_list):\n  if os.path.exists('./medium_15287/' + train + '.jpg'):  \n    img_path = glob('./medium_15287/' + train + '.jpg')[0]\n    shutil.copyfile(img_path, './train/' + train + '.jpg')\n  else:\n    print(train)\n"

In [4]:
print(f'test len: {len(os.listdir("./test"))}')
print(f'train len: {len(os.listdir("./train"))}')

test len: 1529
train len: 13758


In [5]:
train_views=(train_views-np.mean(train_views))/np.std(train_views)
print(train_views.shape)

test_views=(test_views-np.mean(test_views))/np.std(test_views)
print(test_views.shape)

(13758,)
(1529,)


In [8]:
from PIL import Image
from torch.utils.data import Dataset, DataLoader


class SampleDataset(Dataset):
    def __init__(self, phase='test'):
        # self.path = './sample_data'
        self.phase = phase
        
        self.img_list = glob(self.phase + '/*')

        self.transform = transform=transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Resize((180,320), antialias=True),
                               transforms.Pad(padding=(0, 140), padding_mode='reflect'),
                               transforms.Resize((224, 224), antialias=True)
                         ])
    
    def __len__(self):
        return len(self.img_list)
    
    def __getitem__(self, idx):
        img_path = self.img_list[idx]

        img = Image.open(img_path)
        img = self.transform(img)

        vid = img_path.split('/')[-1][:-4]
        views=0
        if self.phase=="test":
            views=test_views[idx]
        else:
            views=train_views[idx]
        
        return img, vid, views

In [9]:
# Prepare sample input data.

batch_size = 64

test_dataset = SampleDataset(phase='test')
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=40)

train_dataset = SampleDataset(phase='train')
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False, num_workers=40)

In [10]:
sample=iter(train_dataloader)
img, vid, views=next(sample)
print(img[0])
print(vid[0], views[0])

tensor([[[0.9694, 0.9821, 0.9888,  ..., 0.9144, 0.8754, 0.8657],
         [0.9952, 0.9892, 0.9779,  ..., 0.8600, 0.7740, 0.6905],
         [0.8666, 0.7739, 0.6570,  ..., 0.5535, 0.4480, 0.3702],
         ...,
         [0.7195, 0.7526, 0.8177,  ..., 0.3175, 0.3325, 0.3279],
         [0.6894, 0.7431, 0.9138,  ..., 0.2987, 0.3502, 0.3404],
         [0.7049, 0.7226, 0.7907,  ..., 0.3351, 0.3166, 0.3465]],

        [[0.3721, 0.3869, 0.3909,  ..., 0.6986, 0.6914, 0.7082],
         [0.3757, 0.3834, 0.3950,  ..., 0.6919, 0.6135, 0.5401],
         [0.3835, 0.3123, 0.2354,  ..., 0.4575, 0.3525, 0.2713],
         ...,
         [0.7074, 0.7247, 0.7555,  ..., 0.3762, 0.3960, 0.3937],
         [0.6747, 0.7126, 0.8514,  ..., 0.3551, 0.4122, 0.4042],
         [0.6880, 0.6891, 0.7238,  ..., 0.3880, 0.3784, 0.4109]],

        [[0.3939, 0.4086, 0.4174,  ..., 0.3540, 0.3901, 0.4149],
         [0.4267, 0.4292, 0.4363,  ..., 0.5980, 0.5611, 0.5182],
         [0.4215, 0.3443, 0.2592,  ..., 0.4640, 0.3337, 0.

In [11]:
#train encoder
from efficientnet_pytorch import EfficientNet
efficientnet = EfficientNet.from_pretrained('efficientnet-b0')
in_features=efficientnet._fc.in_features
print(efficientnet._fc)
efficientnet._fc=nn.Sequential(
    nn.Linear(in_features,100,bias=True),
    nn.ReLU(),
    nn.Linear(100,1)    
)
print(efficientnet._fc)

      
layer=0
for child in efficientnet.children():
    if layer<7:
        for param in child.parameters():
            param.requires_grad=False
    layer+=1


Loaded pretrained weights for efficientnet-b0
Linear(in_features=1280, out_features=1000, bias=True)
Sequential(
  (0): Linear(in_features=1280, out_features=100, bias=True)
  (1): ReLU()
  (2): Linear(in_features=100, out_features=1, bias=True)
)


In [12]:
activation={}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

efficientnet._fc[0].register_forward_hook(get_activation('fc'))

In [13]:
sample=iter(train_dataloader)
batch=next(sample)
print(batch[0].shape)
efficientnet(batch[0])
tmp=activation['fc']

print(tmp, tmp.shape)

torch.Size([64, 3, 224, 224])
tensor([[ 0.1377, -0.0235,  0.1240,  ..., -0.2328, -0.2142,  0.1410],
        [-0.0212, -0.1182, -0.0881,  ...,  0.2749,  0.0510, -0.2523],
        [-0.0353,  0.3424,  0.0714,  ..., -0.0347, -0.1519, -0.4871],
        ...,
        [-0.2779,  0.1164, -0.2016,  ...,  0.0545,  0.0324, -0.1957],
        [-0.3449,  0.0016,  0.3101,  ...,  0.7043, -0.3968,  0.1066],
        [-0.2233,  0.0491, -0.0651,  ..., -0.1581, -0.2054, -0.0356]]) torch.Size([64, 100])


In [14]:
loss_f=nn.MSELoss()
params_to_update=[]
for name, param in efficientnet.named_parameters():
    if param.requires_grad==True:
        params_to_update.append(param)
        print(name, param)
        
optim=torch.optim.SGD(params_to_update, lr=0.0001, momentum=0.8)

_fc.0.weight Parameter containing:
tensor([[ 0.0087,  0.0118, -0.0184,  ...,  0.0154,  0.0194, -0.0065],
        [ 0.0233,  0.0211, -0.0115,  ..., -0.0247,  0.0083,  0.0003],
        [ 0.0269,  0.0268,  0.0121,  ...,  0.0160,  0.0141, -0.0192],
        ...,
        [ 0.0268, -0.0225, -0.0260,  ...,  0.0097,  0.0092,  0.0256],
        [ 0.0275,  0.0163,  0.0048,  ...,  0.0272, -0.0039,  0.0226],
        [ 0.0001,  0.0092,  0.0204,  ...,  0.0037,  0.0110,  0.0168]],
       requires_grad=True)
_fc.0.bias Parameter containing:
tensor([ 0.0020,  0.0262, -0.0011, -0.0017,  0.0271, -0.0242, -0.0144, -0.0256,
        -0.0046, -0.0270, -0.0238, -0.0166,  0.0057, -0.0206, -0.0267, -0.0047,
        -0.0094, -0.0184, -0.0121, -0.0029,  0.0187,  0.0010, -0.0072, -0.0210,
        -0.0069, -0.0007, -0.0218, -0.0268,  0.0238, -0.0237, -0.0246, -0.0221,
        -0.0127,  0.0061, -0.0031,  0.0145, -0.0140,  0.0213, -0.0109, -0.0243,
         0.0119, -0.0211,  0.0226,  0.0153,  0.0262,  0.0212,  0.0084, 

In [15]:
epochs=5

for epoch in range(epochs):    
    efficientnet.train()
    with tqdm(train_dataloader, unit="batch") as batch:
        for img, vid, views in batch:
            batch.set_description(f"Epoch {epoch+1}")
            preds=efficientnet(img).flatten()
            
            batch_loss=loss_f(preds.to(torch.float32),views.to(torch.float32))
            
            optim.zero_grad()
            batch_loss.backward()
            optim.step()
            batch.set_postfix(loss=batch_loss.item())
        
        

Epoch 1:  19%|█▉        | 41/215 [01:03<04:24,  1.52s/batch, loss=0.402] 

In [ ]:
save_data = {}

with torch.no_grad():
  for img, vid, views in tqdm(train_dataloader):
    
    out = efficientnet(img)
    # print(features.shape)
    act=activation['fc']

    print(act, act.shape)
    for b in range(batch_size):
      if b < len(vid):
        save_data[vid[b]] = act[b]

print(f'train len: {len(train_list)}')
print(f'data len: {len(save_data)}')

with open('train.pickle', 'wb') as f:
  pickle.dump(save_data, f, pickle.HIGHEST_PROTOCOL)

데이터 로더 안쓰는 버전

In [ ]:
# # without dataloader ver

# from PIL import Image
# import pickle

# save_data = {}

# transform = transform=transforms.Compose([
#                                transforms.ToTensor(),
#                                transforms.Pad(padding=(0, 140), padding_mode='reflect'),
#                                transforms.Resize((224, 224))
#                       ])

# for vid in tqdm(test_list):
#   img_path = './test/' + vid + '.jpg'

#   img = Image.open(img_path)
#   img = transform(img).unsqueeze(0)

#   # print(img.shape)

#   features = efficientnet.extract_features(img)
#   # print(features.shape)

#   save_data[vid] = features

# with open('test.pickle', 'wb') as f:
#   pickle.dump(save_data, f, pickle.HIGHEST_PROTOCOL)